In [1]:
import os
import cv2
import math
import random
import numpy as np
import datetime as dt
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam

In [2]:
# Specify the height and width to which each video frame will be resized in our dataset.
IMAGE_HEIGHT , IMAGE_WIDTH = 128, 128

# Specify the number of frames of a video that will be fed to the model as one sequence.
SEQUENCE_LENGTH = 30

# Specify the directory containing the UCF50 dataset. 
DATASET_DIR = "Frame_data"

# Thirty videos worth of data
no_sequences = 50

# Specify the list containing the names of the classes used for training. Feel free to choose any set of classes.
CLASSES_LIST = ['Hi', 'I am', 'From', 'Singapore', 'Paiseh', 'I', 'Cannot', 'Speak', 'But', 'Machine', 'Learning', 'Can', 'Help', 'Translate']

In [3]:
label_map = {label:num for num, label in enumerate(CLASSES_LIST)}

In [4]:
def create_dataset():
    '''
    This function will extract the data of the selected classes and create the required dataset.
    Returns:
        features:          A list containing the extracted frames of the videos.
        labels:            A list containing the indexes of the classes associated with the videos.
        video_files_paths: A list containing the paths of the videos in the disk.
    '''

    # Declared Empty Lists to store the features, labels and video file path values.
    features = []
    labels = []
    video_files_paths = []
    
    # Iterating through all the classes mentioned in the classes list
    for class_index, class_name in enumerate(CLASSES_LIST):
        
        # Display the name of the class whose data is being extracted.
        print(f'Extracting Data of Class: {class_name}')
        
        # Get the list of video files present in the specific class name directory.
        files_list = os.listdir(os.path.join(DATASET_DIR, class_name))
        
        # Iterate through all the files present in the files list.
        for file_name in range(no_sequences):
            window = []
            for frame_num in range(SEQUENCE_LENGTH):
            
                # Get the complete video path.
                video_file_path = os.path.join(DATASET_DIR, class_name, str(file_name), str(frame_num)+".npy")

                # Extract the frames of the video file.
                frames = np.load(video_file_path) /255
                window.append(frames)

                # Check if the extracted frames are equal to the SEQUENCE_LENGTH specified above.
                # So ignore the vides having frames less than the SEQUENCE_LENGTH.

                # Append the data to their repective lists.
            features.append(window)
            labels.append(label_map[class_name])
            #video_files_paths.append(video_file_path)

    # Converting the list to numpy arrays
    features = np.asarray(features)
    labels = np.array(labels)  
    
    # Return the frames, class index, and video file path.
    return features, labels

In [5]:
features, labels = create_dataset()
one_hot_encoded_labels = to_categorical(labels)

Extracting Data of Class: Hi
Extracting Data of Class: I am
Extracting Data of Class: From
Extracting Data of Class: Singapore
Extracting Data of Class: Paiseh
Extracting Data of Class: I
Extracting Data of Class: Cannot
Extracting Data of Class: Speak
Extracting Data of Class: But
Extracting Data of Class: Machine
Extracting Data of Class: Learning
Extracting Data of Class: Can
Extracting Data of Class: Help
Extracting Data of Class: Translate


In [6]:
def create_LRCN_model():
    '''
    This function will construct the required LRCN model.
    Returns:
        model: It is the required constructed LRCN model.
    '''

    # We will use a Sequential model for model construction.
    model = Sequential()
    
    # Define the Model Architecture.
    ########################################################################################################################
    
    model.add(TimeDistributed(Conv2D(16, (3, 3), padding='same',activation = 'relu'),
                              input_shape = (SEQUENCE_LENGTH, IMAGE_HEIGHT, IMAGE_WIDTH, 3)))
    
    model.add(TimeDistributed(MaxPooling2D((4, 4)))) 
    model.add(TimeDistributed(Dropout(0.20)))
    
    model.add(TimeDistributed(Conv2D(32, (3, 3), padding='same',activation = 'relu')))  # dropouts changed to 0.2
    model.add(TimeDistributed(MaxPooling2D((4, 4))))
    model.add(TimeDistributed(Dropout(0.20)))
    
    model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same',activation = 'relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Dropout(0.20)))
    
    model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same',activation = 'relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    #model.add(TimeDistributed(Dropout(0.25)))
                                      
    model.add(TimeDistributed(Flatten()))
                                      
    model.add(LSTM(32))
                                      
    model.add(Dense(len(CLASSES_LIST), activation = 'softmax'))

    ########################################################################################################################

    # Display the models summary.
    model.summary()
    
    # Return the constructed LRCN model.
    return model

In [19]:
accuracies = []
f1_scores = []
precisions = []
recalls = []

kf = KFold(n_splits=5, shuffle=True)
for train_index, test_index in kf.split(features):
    
    np.random.shuffle(train_index)
    np.random.shuffle(test_index)
    
    features_train, features_test = features[train_index], features[test_index]
    labels_train, labels_test = one_hot_encoded_labels[train_index], one_hot_encoded_labels[test_index]
    
    # Construct the required LRCN model.
    LRCN_model = create_LRCN_model()

    # Display the success message.
    print("Model Created Successfully!")
    
    # Create an Instance of Early Stopping Callback.
    early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 15, mode = 'min', restore_best_weights = True)

    # Compile the model and specify loss function, optimizer and metrics to the model.
    LRCN_model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

    # Start training the model.
    LRCN_model_training_history = LRCN_model.fit(x = features_train, y = labels_train, epochs = 200, batch_size = 4 ,
                                                 shuffle = True, validation_split = 0.2, callbacks = [early_stopping_callback])
    
    # Evaluate the trained model.
    predictions = LRCN_model.predict(features_test)
    y_true = np.argmax(labels_test, axis=1).tolist()
    y_hat = np.argmax(predictions, axis=1).tolist()
    conf = multilabel_confusion_matrix(y_true,y_hat)
    acc = accuracy_score(y_true, y_hat)
    
    indiv_f1 = []
    indiv_pre = []
    indiv_re = []
    for matrix in conf:
        FP = matrix[0][1]
        FN = matrix[1][0]
        TP = matrix[1][1]
        precision = TP/(TP+FP)
        recall = TP/(TP+FN)
        indiv_f1.append((2*precision*recall)/(precision+recall))
        indiv_pre.append(precision)
        indiv_re.append(recall)
    f1 = np.nansum(indiv_f1)/len(indiv_f1)
    pre = np.nansum(indiv_pre)/len(indiv_pre)
    re = np.nansum(indiv_re)/len(indiv_re)
    
    accuracies.append(acc)
    f1_scores.append(f1)
    precisions.append(pre)
    recalls.append(re)
    
    print(acc)
    print(f1)
    
    # check cat_accuracy
    LRCN_model.evaluate(features_test, labels_test)

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_156 (TimeD  (None, 30, 128, 128, 16)  448      
 istributed)                                                     
                                                                 
 time_distributed_157 (TimeD  (None, 30, 32, 32, 16)   0         
 istributed)                                                     
                                                                 
 time_distributed_158 (TimeD  (None, 30, 32, 32, 16)   0         
 istributed)                                                     
                                                                 
 time_distributed_159 (TimeD  (None, 30, 32, 32, 32)   4640      
 istributed)                                                     
                                                                 
 time_distributed_160 (TimeD  (None, 30, 8, 8, 32)   

Epoch 36/200
112/112 [==============================] - 31s 278ms/step - loss: 0.1805 - accuracy: 0.9330 - val_loss: 0.3134 - val_accuracy: 0.9196
Epoch 37/200
112/112 [==============================] - 32s 282ms/step - loss: 0.1914 - accuracy: 0.9487 - val_loss: 0.3879 - val_accuracy: 0.8750
Epoch 38/200
112/112 [==============================] - 31s 276ms/step - loss: 0.1239 - accuracy: 0.9732 - val_loss: 0.1702 - val_accuracy: 0.9464
Epoch 39/200
112/112 [==============================] - 30s 271ms/step - loss: 0.0615 - accuracy: 0.9911 - val_loss: 0.2741 - val_accuracy: 0.9107
Epoch 40/200
112/112 [==============================] - 31s 275ms/step - loss: 0.0627 - accuracy: 0.9866 - val_loss: 0.3502 - val_accuracy: 0.8661
Epoch 41/200
112/112 [==============================] - 31s 280ms/step - loss: 0.2142 - accuracy: 0.9263 - val_loss: 0.4698 - val_accuracy: 0.8750
Epoch 42/200
112/112 [==============================] - 31s 274ms/step - loss: 0.1814 - accuracy: 0.9509 - val_loss: 0

Epoch 8/200
112/112 [==============================] - 33s 296ms/step - loss: 2.6383 - accuracy: 0.0826 - val_loss: 2.6384 - val_accuracy: 0.0446
Epoch 9/200
112/112 [==============================] - 32s 287ms/step - loss: 2.6324 - accuracy: 0.0804 - val_loss: 2.6248 - val_accuracy: 0.0982
Epoch 10/200
112/112 [==============================] - 32s 290ms/step - loss: 2.5884 - accuracy: 0.1049 - val_loss: 2.4926 - val_accuracy: 0.1875
Epoch 11/200
112/112 [==============================] - 33s 293ms/step - loss: 2.4824 - accuracy: 0.1629 - val_loss: 2.3756 - val_accuracy: 0.2321
Epoch 12/200
112/112 [==============================] - 33s 291ms/step - loss: 2.2977 - accuracy: 0.1853 - val_loss: 2.1841 - val_accuracy: 0.2232
Epoch 13/200
112/112 [==============================] - 32s 282ms/step - loss: 2.0517 - accuracy: 0.2835 - val_loss: 2.0518 - val_accuracy: 0.2946
Epoch 14/200
112/112 [==============================] - 32s 290ms/step - loss: 1.8900 - accuracy: 0.3348 - val_loss: 1.8

Epoch 64/200
112/112 [==============================] - 32s 290ms/step - loss: 0.1247 - accuracy: 0.9732 - val_loss: 0.2255 - val_accuracy: 0.9107
Epoch 65/200
112/112 [==============================] - 32s 289ms/step - loss: 0.1026 - accuracy: 0.9821 - val_loss: 0.2078 - val_accuracy: 0.9286
Epoch 66/200
112/112 [==============================] - 31s 275ms/step - loss: 0.1609 - accuracy: 0.9598 - val_loss: 0.2687 - val_accuracy: 0.9196
Epoch 67/200
112/112 [==============================] - 30s 269ms/step - loss: 0.1091 - accuracy: 0.9821 - val_loss: 0.1475 - val_accuracy: 0.9375
Epoch 68/200
112/112 [==============================] - 32s 288ms/step - loss: 0.0663 - accuracy: 0.9911 - val_loss: 0.1085 - val_accuracy: 0.9732
Epoch 69/200
112/112 [==============================] - 32s 284ms/step - loss: 0.0965 - accuracy: 0.9844 - val_loss: 0.1080 - val_accuracy: 0.9643
Epoch 70/200
112/112 [==============================] - 30s 271ms/step - loss: 0.0702 - accuracy: 0.9866 - val_loss: 0

Epoch 28/200
112/112 [==============================] - 32s 283ms/step - loss: 0.1039 - accuracy: 0.9799 - val_loss: 0.5708 - val_accuracy: 0.8125
Epoch 29/200
112/112 [==============================] - 33s 292ms/step - loss: 0.1719 - accuracy: 0.9621 - val_loss: 0.1994 - val_accuracy: 0.9375
Epoch 30/200
112/112 [==============================] - 32s 284ms/step - loss: 0.0738 - accuracy: 0.9888 - val_loss: 0.1979 - val_accuracy: 0.9554
Epoch 31/200
112/112 [==============================] - 31s 279ms/step - loss: 0.0973 - accuracy: 0.9821 - val_loss: 0.3281 - val_accuracy: 0.9018
Epoch 32/200
112/112 [==============================] - 33s 296ms/step - loss: 0.1480 - accuracy: 0.9464 - val_loss: 0.2657 - val_accuracy: 0.9464
Epoch 33/200
112/112 [==============================] - 33s 295ms/step - loss: 0.2076 - accuracy: 0.9509 - val_loss: 0.3448 - val_accuracy: 0.8839
Epoch 34/200
112/112 [==============================] - 32s 288ms/step - loss: 0.0797 - accuracy: 0.9888 - val_loss: 0

Epoch 1/200
112/112 [==============================] - 37s 309ms/step - loss: 2.6509 - accuracy: 0.0625 - val_loss: 2.6420 - val_accuracy: 0.0625
Epoch 2/200
112/112 [==============================] - 34s 301ms/step - loss: 2.6422 - accuracy: 0.0893 - val_loss: 2.6468 - val_accuracy: 0.0357
Epoch 3/200
112/112 [==============================] - 33s 299ms/step - loss: 2.6370 - accuracy: 0.0871 - val_loss: 2.6469 - val_accuracy: 0.1161
Epoch 4/200
112/112 [==============================] - 34s 305ms/step - loss: 2.5661 - accuracy: 0.1384 - val_loss: 2.6058 - val_accuracy: 0.0804
Epoch 5/200
112/112 [==============================] - 33s 299ms/step - loss: 2.2325 - accuracy: 0.2321 - val_loss: 2.2045 - val_accuracy: 0.1964
Epoch 6/200
112/112 [==============================] - 34s 300ms/step - loss: 1.9709 - accuracy: 0.3103 - val_loss: 1.8604 - val_accuracy: 0.2946
Epoch 7/200
112/112 [==============================] - 34s 305ms/step - loss: 1.7249 - accuracy: 0.3750 - val_loss: 1.7183 -

Epoch 57/200
112/112 [==============================] - 35s 314ms/step - loss: 0.0031 - accuracy: 1.0000 - val_loss: 0.0527 - val_accuracy: 0.9821
Epoch 58/200
112/112 [==============================] - 35s 310ms/step - loss: 0.0027 - accuracy: 1.0000 - val_loss: 0.0644 - val_accuracy: 0.9821
Epoch 59/200
112/112 [==============================] - 33s 293ms/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 0.0562 - val_accuracy: 0.9821
Epoch 60/200
112/112 [==============================] - 33s 297ms/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 0.0563 - val_accuracy: 0.9821
Epoch 61/200
112/112 [==============================] - 35s 314ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.0783 - val_accuracy: 0.9732
Epoch 62/200
112/112 [==============================] - 33s 298ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.0826 - val_accuracy: 0.9732
Epoch 63/200
112/112 [==============================] - 32s 290ms/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0

Epoch 23/200
112/112 [==============================] - 34s 307ms/step - loss: 0.2421 - accuracy: 0.9375 - val_loss: 0.4076 - val_accuracy: 0.8839
Epoch 24/200
112/112 [==============================] - 33s 290ms/step - loss: 0.1949 - accuracy: 0.9576 - val_loss: 0.3273 - val_accuracy: 0.8661
Epoch 25/200
112/112 [==============================] - 32s 282ms/step - loss: 0.2912 - accuracy: 0.9107 - val_loss: 0.5908 - val_accuracy: 0.7679
Epoch 26/200
112/112 [==============================] - 33s 294ms/step - loss: 0.3738 - accuracy: 0.8772 - val_loss: 0.2986 - val_accuracy: 0.9107
Epoch 27/200
112/112 [==============================] - 33s 297ms/step - loss: 0.2154 - accuracy: 0.9464 - val_loss: 0.3133 - val_accuracy: 0.9107
Epoch 28/200
112/112 [==============================] - 33s 291ms/step - loss: 0.1807 - accuracy: 0.9509 - val_loss: 0.2115 - val_accuracy: 0.9554
Epoch 29/200
112/112 [==============================] - 32s 284ms/step - loss: 0.1319 - accuracy: 0.9777 - val_loss: 0

Epoch 79/200
112/112 [==============================] - 32s 287ms/step - loss: 0.0051 - accuracy: 1.0000 - val_loss: 0.0373 - val_accuracy: 0.9821
Epoch 80/200
112/112 [==============================] - 33s 291ms/step - loss: 0.0044 - accuracy: 1.0000 - val_loss: 0.0347 - val_accuracy: 0.9911
Epoch 81/200
112/112 [==============================] - 34s 301ms/step - loss: 0.0049 - accuracy: 1.0000 - val_loss: 0.0387 - val_accuracy: 0.9911
Epoch 82/200
112/112 [==============================] - 33s 296ms/step - loss: 0.0040 - accuracy: 1.0000 - val_loss: 0.0420 - val_accuracy: 0.9911
Epoch 83/200
112/112 [==============================] - 32s 283ms/step - loss: 0.0039 - accuracy: 1.0000 - val_loss: 0.0439 - val_accuracy: 0.9911
Epoch 84/200
112/112 [==============================] - 33s 290ms/step - loss: 0.0034 - accuracy: 1.0000 - val_loss: 0.0430 - val_accuracy: 0.9911
Epoch 85/200
112/112 [==============================] - 34s 302ms/step - loss: 0.0032 - accuracy: 1.0000 - val_loss: 0

In [20]:
print("Accuracies:", accuracies)
print("F1 Scores:", f1_scores)
print("Precisions:", precisions)
print("Recalls:", recalls)

Accuracies: [0.9785714285714285, 0.9857142857142858, 0.9571428571428572, 0.9785714285714285, 0.9642857142857143]
F1 Scores: [0.9765359675134111, 0.982553000859637, 0.9564249251082637, 0.9793357342937176, 0.9638251261567159]
Precisions: [0.9788265306122449, 0.9806122448979593, 0.9631519274376419, 0.9790764790764791, 0.9662958773252892]
Recalls: [0.9768089053803338, 0.9869047619047618, 0.9566326530612245, 0.9808691308691309, 0.9652014652014652]


In [21]:
print("Mean Accuracy:", np.nansum(accuracies)/len(accuracies))
print("Mean F1 Score:", np.nansum(f1_scores)/len(f1_scores))
print("Mean Precision:", np.nansum(precisions)/len(precisions))
print("Mean Recall:", np.nansum(recalls)/len(recalls))

Mean Accuracy: 0.972857142857143
Mean F1 Score: 0.9717349507863491
Mean Precision: 0.973592611869923
Mean Recall: 0.9732833832833832
